In [1]:
!pip install wallstreet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Imports
import numpy as np
import pandas as pd
from scipy.stats import norm
from datetime import datetime
from wallstreet import Stock, Call, Put
import concurrent.futures

In [3]:
# Variables
ticker = "AAPL"
risk_free_rate = 0.034

In [4]:
def black_scholes(stock_price, strike_price, time_to_expiry, risk_free_rate, volatility, option_type):
    d1 = (np.log(stock_price / strike_price) + (risk_free_rate + 0.5 * volatility ** 2) * time_to_expiry) / (volatility * np.sqrt(time_to_expiry))
    d2 = d1 - volatility * np.sqrt(time_to_expiry)
    
    if option_type == "call":
        option_value = stock_price * norm.cdf(d1) - strike_price * np.exp(-risk_free_rate * time_to_expiry) * norm.cdf(d2)
    elif option_type == "put":
        option_value = strike_price * np.exp(-risk_free_rate * time_to_expiry) * norm.cdf(-d2) - stock_price * norm.cdf(-d1)
    else:
        raise ValueError("Invalid option type: must be 'call' or 'put'")

    return option_value

def filter_positive_value_diff(options_df):
    return options_df[options_df["value_diff"] > 0]

def process_expiration_date(date):
    call_option = Call(ticker, d=date.day, m=date.month, y=date.year)
    put_option = Put(ticker, d=date.day, m=date.month, y=date.year)
    
    calls = []
    puts = []
    
    for strike in call_option.strikes:
        call_option.set_strike(strike)
        implied_volatility = call_option.implied_volatility()
        expiration_date = datetime.strptime(call_option.expiration, "%d-%m-%Y")
        time_to_expiry = (expiration_date - datetime.today()).days / 365
        calls.append({
            "Contract Name": call_option.code,
            "Strike": strike,
            "Last Price": call_option.price,
            "time_to_expiry": time_to_expiry,
            "Implied Volatility": implied_volatility,
            "bs_value": black_scholes(stock_price, strike, time_to_expiry, risk_free_rate, implied_volatility, "call"),
            "value_diff": None
        })

    for strike in put_option.strikes:
        put_option.set_strike(strike)
        implied_volatility = put_option.implied_volatility()
        expiration_date = datetime.strptime(put_option.expiration, "%d-%m-%Y")
        time_to_expiry = (expiration_date - datetime.today()).days / 365
        puts.append({
            "Contract Name": put_option.code,
            "Strike": strike,
            "Last Price": put_option.price,
            "time_to_expiry": time_to_expiry,
            "Implied Volatility": implied_volatility,
            "bs_value": black_scholes(stock_price, strike, time_to_expiry, risk_free_rate, implied_volatility, "put"),
            "value_diff": None
        })

    return calls, puts

In [5]:
# Get the stock price for the most recent trading day
s = Stock(ticker)
stock_price = s.price

# Get list of expiration dates
g = Call(ticker)
expiration_dates = g.expirations
date_objects = [datetime.strptime(date_str, "%d-%m-%Y") for date_str in expiration_dates]

# Initialize empty lists for calls and puts
all_calls = []
all_puts = []

No options listed for given date, using 05-05-2023 instead


In [6]:
# Loop through each expiration date using concurrent.futures for parallelism
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = [executor.submit(process_expiration_date, date) for date in date_objects]
    
    for future in concurrent.futures.as_completed(futures):
        calls, puts = future.result()
        all_calls.extend(calls)
        all_puts.extend(puts)


/usr/local/lib/python3.10/dist-packages/scipy/optimize/_minpack_py.py:178: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
/usr/local/lib/python3.10/dist-packages/scipy/optimize/_minpack_py.py:178: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)


In [7]:
all_calls = pd.DataFrame(all_calls)
all_puts = pd.DataFrame(all_puts)

all_calls["value_diff"] = all_calls["bs_value"] - all_calls["Last Price"]
all_puts["value_diff"] = all_puts["bs_value"] - all_puts["Last Price"]

print("Call Options:")
print(all_calls[["Contract Name", "Strike", "Last Price", "bs_value", "value_diff"]])
print("\nPut Options:")
print(all_puts[["Contract Name", "Strike", "Last Price", "bs_value", "value_diff"]])

Call Options:
           Contract Name  Strike  Last Price   bs_value  value_diff
0    AAPL230609C00100000   100.0       67.92  68.484064    0.564064
1    AAPL230609C00135000   135.0       32.18  33.604487    1.424487
2    AAPL230609C00140000   140.0       28.99  28.874921   -0.115079
3    AAPL230609C00150000   150.0       21.45  21.277707   -0.172293
4    AAPL230609C00155000   155.0       15.49  15.345471   -0.144529
..                   ...     ...         ...        ...         ...
920  AAPL251219C00270000   270.0        7.00   7.031730    0.031730
921  AAPL251219C00280000   280.0        6.00   6.025882    0.025882
922  AAPL251219C00290000   290.0        4.75   4.767107    0.017107
923  AAPL251219C00300000   300.0        3.95   3.962263    0.012263
924  AAPL251219C00310000   310.0        3.35   3.359027    0.009027

[925 rows x 5 columns]

Put Options:
           Contract Name  Strike  Last Price    bs_value  value_diff
0    AAPL230609P00095000    95.0        0.02    0.017692   -0.0

In [8]:
# Write calls and puts to separate sheets in a single Excel file
with pd.ExcelWriter(ticker + '_options_data.xlsx', engine='openpyxl') as writer:
    all_calls.to_excel(writer, sheet_name='Calls', index=False)
    all_puts.to_excel(writer, sheet_name='Puts', index=False)

In [9]:
# Find most undervalued call and put options
positive_value_diff_calls = filter_positive_value_diff(all_calls)
positive_value_diff_puts = filter_positive_value_diff(all_puts)

In [10]:
try:
    most_undervalued_call_idx = positive_value_diff_calls['value_diff'].idxmax()
    most_undervalued_call = positive_value_diff_calls.loc[most_undervalued_call_idx]
    print("Most undervalued call option:")
    print(most_undervalued_call[['Contract Name', 'Strike', 'Last Price', 'time_to_expiry', 'bs_value', 'value_diff']])
except ValueError:
    print("No undervalued call options found.")

try:
    most_undervalued_put_idx = positive_value_diff_puts['value_diff'].idxmax()
    most_undervalued_put = positive_value_diff_puts.loc[most_undervalued_put_idx]
    print("\nMost undervalued put option:")
    print(most_undervalued_put[['Contract Name', 'Strike', 'Last Price', 'time_to_expiry', 'bs_value', 'value_diff']])
except ValueError:
    print("No undervalued put options found.")

print("\n\nOptions data has been saved to '" + ticker +  "_options_data.xlsx'\n\n")

Most undervalued call option:
Contract Name     AAPL230519C00065000
Strike                           65.0
Last Price                      67.15
time_to_expiry               0.043836
bs_value                   103.236805
value_diff                  36.086805
Name: 120, dtype: object

Most undervalued put option:
Contract Name     AAPL240621P00320000
Strike                          320.0
Last Price                     156.42
time_to_expiry               1.136986
bs_value                   159.982858
value_diff                   3.562858
Name: 690, dtype: object


Options data has been saved to 'AAPL_options_data.xlsx'


